In [1]:
import requests
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def lohnsteuer(brutto, stkl=1):
    brutto_ct = int(brutto * 100)
    url = f"https://www.bmf-steuerrechner.de/interface/2023Version1.xhtml?code=ext2023&LZZ=1&RE4={brutto_ct}&STKL={stkl}"
    r = requests.get(url)
    root = ET.fromstring(r.text)
    lz = root.find("ausgaben/ausgabe[@name='LSTLZZ']")
    sol = root.find("ausgaben/ausgabe[@name='SOLZLZZ']")
    return {
        "lohnsteuer": int(lz.attrib["value"]) / 100,
        "solz": int(sol.attrib["value"]) / 100,
    }

In [3]:
def rv(brutto):
    return min(7300*12, brutto) * 9.3 * 0.01

def av(brutto):
    return min(7300*12, brutto) * 1.3 * 0.01

def kv(brutto, zkv=1.2):
    return min(59850, brutto) * (zkv+14.6)/2 * 0.01

def pv(brutto, kinderlos=True):
    return min(59850, brutto) * (3.05+0.35*kinderlos)/2 * 0.01

In [4]:
def netto(brutto, zkv=1.2, stkl=1, kinderlos=True):
    _rv = rv(brutto)
    _kv = kv(brutto, zkv)
    _pv = pv(brutto, kinderlos)
    _av = av(brutto)
    st = lohnsteuer(brutto, stkl)
    _lst = st["lohnsteuer"]
    _sol = st["solz"]
    return {
        "brutto": brutto,
        "rv": _rv,
        "av": _av,
        "lst": _lst,
        "solz": _sol,
        "kv": _kv,
        "pv": _pv,
        "netto": brutto - _rv - _kv - _pv - _av - _lst - _sol,
    }

In [5]:
netto(250000)

{'brutto': 250000,
 'rv': 8146.800000000001,
 'av': 1138.8,
 'lst': 88930.0,
 'solz': 4891.15,
 'kv': 4728.15,
 'pv': 1017.45,
 'netto': 141147.65000000002}

In [6]:
startalter = 24
endalter = 100
inflation = 1.02
rendite = 1.04
rente = 2000
rentenalter = 63
startkap = 40000
sparrate = 0.20

def ek_m(age):
    if age < 27:
        return 4000
    elif age < 50:
        return 6000
    else:
        return 8000

ek_j = lambda ek_m: ek_m * 12

In [7]:
kap = startkap
work = []
for age in np.arange(startalter, rentenalter):
    kap *= rendite
    ekm = ek_m(age)
    ekj = ek_j(ekm)
    abgaben = netto(ekj)
    sparen = ekj * sparrate
    rest = abgaben["netto"] - sparen
    rest_infl = rest * pow(inflation, age-startalter)
    kap += rest_infl * sparrate
    work += [{
        "Alter": age,
        "Kapital": kap,
        "Brutto (m)": ekm,
        "Abgaben (m)": ekm-abgaben["netto"]/12,
        "Sparen (m)": sparen / 12,
        "Rest (m)": rest / 12,
        "Rest infl (m)": rest_infl / 12,
    }]

In [8]:
kap = work[-1]["Kapital"]
retire = []
for age in np.arange(rentenalter, endalter):
    kap *= rendite
    rente_jinfl = rente * 12 * pow(inflation, age-startalter)
    kap -= rente_jinfl
    retire += [{
        "Alter": age,
        "Kapital": kap,
        "Brutto (m)": rente,
        "Abgaben (m)": 0,
        "Sparen (m)": 0,
        "Rest (m)": rente,
        "Rest infl (m)": rente_jinfl / 12,
    }]

In [9]:
for row in work:
    print(row)

{'Alter': 24, 'Kapital': 45894.8, 'Brutto (m)': 4000, 'Abgaben (m)': 1410.5, 'Sparen (m)': 800.0, 'Rest (m)': 1789.5, 'Rest infl (m)': 1789.5}
{'Alter': 25, 'Kapital': 52111.288, 'Brutto (m)': 4000, 'Abgaben (m)': 1410.5, 'Sparen (m)': 800.0, 'Rest (m)': 1789.5, 'Rest infl (m)': 1825.29}
{'Alter': 26, 'Kapital': 58664.04944, 'Brutto (m)': 4000, 'Abgaben (m)': 1410.5, 'Sparen (m)': 800.0, 'Rest (m)': 1789.5, 'Rest infl (m)': 1861.7957999999999}
{'Alter': 27, 'Kapital': 67253.57073664, 'Brutto (m)': 6000, 'Abgaben (m)': 2348.7999999999997, 'Sparen (m)': 1200.0, 'Rest (m)': 2451.2000000000003, 'Rest infl (m)': 2601.2330496000004}
{'Alter': 28, 'Kapital': 76311.53207152641, 'Brutto (m)': 6000, 'Abgaben (m)': 2348.7999999999997, 'Sparen (m)': 1200.0, 'Rest (m)': 2451.2000000000003, 'Rest infl (m)': 2653.257710592}
{'Alter': 29, 'Kapital': 85859.16822991669, 'Brutto (m)': 6000, 'Abgaben (m)': 2348.7999999999997, 'Sparen (m)': 1200.0, 'Rest (m)': 2451.2000000000003, 'Rest infl (m)': 2706.3228

In [10]:
for row in retire:
    print(row)

{'Alter': 63, 'Kapital': 915722.4539272395, 'Brutto (m)': 2000, 'Abgaben (m)': 0, 'Sparen (m)': 0, 'Rest (m)': 2000, 'Rest infl (m)': 4329.489536499713}
{'Alter': 64, 'Kapital': 899358.4001575726, 'Brutto (m)': 2000, 'Abgaben (m)': 0, 'Sparen (m)': 0, 'Rest (m)': 2000, 'Rest infl (m)': 4416.079327229707}
{'Alter': 65, 'Kapital': 881279.925198584, 'Brutto (m)': 2000, 'Abgaben (m)': 0, 'Sparen (m)': 0, 'Rest (m)': 2000, 'Rest infl (m)': 4504.400913774302}
{'Alter': 66, 'Kapital': 861397.25502193, 'Brutto (m)': 2000, 'Abgaben (m)': 0, 'Sparen (m)': 0, 'Rest (m)': 2000, 'Rest infl (m)': 4594.488932049788}
{'Alter': 67, 'Kapital': 839616.6006945178, 'Brutto (m)': 2000, 'Abgaben (m)': 0, 'Sparen (m)': 0, 'Rest (m)': 2000, 'Rest infl (m)': 4686.378710690784}
{'Alter': 68, 'Kapital': 815839.9893034433, 'Brutto (m)': 2000, 'Abgaben (m)': 0, 'Sparen (m)': 0, 'Rest (m)': 2000, 'Rest infl (m)': 4780.106284904599}
{'Alter': 69, 'Kapital': 789965.0879483488, 'Brutto (m)': 2000, 'Abgaben (m)': 0, 'Sp